# seaMass SWATH demo
This notebook illustrates how to load seaMass input/output on SWATH data. Firstly, we need to import the relevant modules...

In [ ]:
using Plots; plotly(size=(980,350))
using Interact
import SeaMass

In [ ]:
reload("SeaMass")

### 1. Display spectra from an mzMLb file

In [ ]:
mzmlSpectra = SeaMass.MzmlSpectrum[]
maxSpectraIntensity = 0.0
for i = 1:100
    push!(mzmlSpectra, SeaMass.MzmlSpectrum("../data/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500.mzMLb",
        2 + (i - 1) * 51))
    maxIntensity = maximum(mzmlSpectra[i].intensities)
    maxSpectraIntensity = (maxSpectraIntensity > maxIntensity) ? maxSpectraIntensity : maxIntensity
end

@manipulate for i = slider(1:100, label="Spectrum", value=63, continuous_update=true)
    plot(mzmlSpectra[i].mzs, mzmlSpectra[i].intensities, label="mzML input", m=2)
    plot!(xlabel = "m/z (Th)", ylabel = "intensity",
        xlims=(602,605), ylims=(-0.05*maxSpectraIntensity,maxSpectraIntensity))
end

### 2. Compare the spectra from the SMI file

In [ ]:
@manipulate for i = slider(1:100, label="Spectrum", value=63, continuous_update=true)
    smiSpectrum = SeaMass.SmiSpectrum(
        "../data/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500.pos_411.25.smi", i)

    plot(smiSpectrum.locations, vcat(smiSpectrum.counts, 0.0), line=:steppost, label="SMI input", m=2)
    plot!(mzmlSpectra[i].mzs, mzmlSpectra[i].intensities * smiSpectrum.exposure, label="mzML input", m=2)
    plot!(xlabel = "m/z (Th)", ylabel = "ion density",
        xlims=(602,605),
        ylims=(-0.05*maxSpectraIntensity*smiSpectrum.exposure,maxSpectraIntensity*smiSpectrum.exposure))
end

### 3. Compare output of 1D seaMass to 1D SMI file

In [ ]:
smiSpectrum = SeaMass.SmiSpectrum(
    "../data/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500__index3163.pos_411.25.smi", 1)
smiBinWidths = smiSpectrum.locations[2:end] - smiSpectrum.locations[1:end-1]

smoSpectrum = SeaMass.SmoSpectrum(
    "../data/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500__index3163.pos_411.25.smo1D", 1)
binnedSmoSpectrum = SeaMass.BinnedSmoSpectrum(smoSpectrum)
sampledSmoSpectrum = SeaMass.MzmlSpectrum(smoSpectrum, 100000)

plot(smiSpectrum.locations, vcat(smiSpectrum.counts  ./ smiBinWidths, 0.0), line=:steppost, label="SMI input")
plot!(binnedSmoSpectrum.locations, vcat(binnedSmoSpectrum.counts / binnedSmoSpectrum.binWidth, 0.0),
      line=:steppost, label="binned reconstruction")
plot!(sampledSmoSpectrum.mzs, sampledSmoSpectrum.intensities, label="sampled reconstruction")
plot!(xlabel = "m/z (Th)", ylabel = "ion density (counts per Th)")

#### 3.1. Animation

In [ ]:
smoFilenames = filter!(r"^P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500__index3163\.pos_411.25\..*smo1D$",
    readdir("../data/"))

@manipulate for i=slider(0:length(smoFilenames)-1, label="Iteration", value=0,
        continuous_update=true)
    smoSpectrum = SeaMass.SmoSpectrum("../data/" * smoFilenames[i+1], 1)
    sampledSmoSpectrum = SeaMass.MzmlSpectrum(smoSpectrum, 10000)
    
    plot(smiSpectrum.locations, vcat(smiSpectrum.counts  ./ smiBinWidths, 0.0), line=:steppost, label="SMI input")
    plot!(sampledSmoSpectrum.mzs, sampledSmoSpectrum.intensities, label="sampled reconstruction")
    plot!(xlabel = "m/z (Th)", ylabel = "ion density (counts per Th)")
    plot!(xlims = (602.,605.), ylims = (-10000.,120000.))
end

### 4. Compare output of 2D seaMass to 1D SMI file

In [ ]:
smiSpectra = SeaMass.SmiSpectrum[]
maxSpectraBinCounts = 0.0
for i = 1:100
    smiSpectrum = SeaMass.SmiSpectrum(
        "../data/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500.pos_411.25.smi", i)
    smiBinWidths = smiSpectrum.locations[2:end] - smiSpectrum.locations[1:end-1]
    smiSpectrum.counts = vcat(smiSpectrum.counts  ./ smiBinWidths, 0.0)
    maxBinCounts = maximum(smiSpectrum.counts)
    maxSpectraBinCounts = (maxSpectraBinCounts > maxBinCounts) ? maxSpectraBinCounts : maxBinCounts
    push!(smiSpectra, smiSpectrum)
end

@manipulate for i = slider(1:100, label="Spectrum", value=63, continuous_update=true)
    smoSpectrum = SeaMass.SmoSpectrum(
        "../data/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500.pos_411.25.smo1D", i)
    binnedSmoSpectrum = SeaMass.BinnedSmoSpectrum(smoSpectrum)
    sampledSmoSpectrum = SeaMass.MzmlSpectrum(smoSpectrum, 100000)

    plot(smiSpectra[i].locations, smiSpectra[i].counts, line=:steppost, label="SMI input")
    plot!(binnedSmoSpectrum.locations, vcat(binnedSmoSpectrum.counts / binnedSmoSpectrum.binWidth, 0.0),
      line=:steppost, label="binned reconstruction")
    plot!(sampledSmoSpectrum.mzs, sampledSmoSpectrum.intensities, label="sampled reconstruction")
    plot!(xlabel = "m/z (Th)", ylabel = "ion density (counts per Th)",
        xlims=(602,605), ylims=(-0.05*maxSpectraBinCounts,maxSpectraBinCounts))
end